In [28]:
import pandas as pd
import pandas_datareader.data as pdr

from pandas import Series, DataFrame
import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [29]:
stock = "GOOG"

In [30]:
df = pdr.get_data_yahoo("SPY", start="2016-01-01",end="2020-12-31")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed


In [31]:
dfSorted['30MA'] = dfSorted['Close'].rolling(window=30).mean()

In [35]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30MA'],name="30MA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()